<a href="https://colab.research.google.com/github/KSG722/class2022Spring/blob/main/huggingface_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 각 task의 설명 확인하기

https://huggingface.co/tasks <- 여기 들어가서 task의 설명 읽어보기




In [ ]:
!pip install gradio
import gradio as gr

### [Image classification](https://huggingface.co/tasks/image-classification)

직접 링크로 타고 들어가서 설명 읽어보기(시험문제는 이 걸 할줄 안다는 전제로 나올 수도)
이미지를 넣으면 각각의 확률값과 함게 class(ex. 이집트 고양이)가 분류된다.

e.g. https://huggingface.co/google/vit-base-patch16-224 \
How to use

구체적인 image classification 모델

how to use에 적힌 코딩을 긁어오자

In [ ]:
!pip install transformers

In [ ]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

url은 딱봐도 이후 분류할 그림 이미지. 

pertrained는 행렬이자 잘 학습된 딥러닝모델. 2개의 pretrained가 있다.

각각의 이름이나 코딩을 보면 feature_extractor : 사진에서 특징을 찾아내는 행렬. 어두우면 1, 밝으면 0, 이런 식으로 사진에서 중요한 숫자정보를 압축적으로 뽑아냄. object를 정의할 수 있는 핵심feature들을 뽑아냄.

model : 위에서 찾아낸 특징을 가지고 image classfication을 직접 행하는 행렬

전체적으로는
inputs에는 feature_extractor 행렬을 통해 object를 정의할 수 있는 핵심feature들이 뽑아진 image를 넣고(첫번째 행렬이 곱해진 거)

outputs에는 model 행렬을 통해 1000개의 class 중 제일 적절한 class를 도출(두번째 행렬이 곱해진 것).
참고로 1000개의 class 당 확률값이 0.xxx로 나오고, 이를 모두 더하면 1.


마지막 print로 예상된 class(분류)를 도출

즉 url 이미지라는 input에 행렬값들을 곱해서 output으로 class를 분류해 도출해내는 것.

-> 이집션 캣인지 호랑이인지 강아지인지 확률값으로 계산 한 후 가장 확률이 높은 것으로 classify. 이집션 캣이 0.98xx값이 나옴. 참고로 값을 다 더하면 1.


Demo in Gradio

In [ ]:
def func (image):
  feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
  model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

  inputs = feature_extractor(images=image, return_tensors="pt")
  outputs = model(**inputs)
  logits = outputs.logits
  # model predicts one of the 1000 ImageNet classes
  predicted_class_idx = logits.argmax(-1).item()
  predicted_class = model.config.id2label[predicted_class_idx]
  return predicted_class

위의 함수를 가져와서 func이라는 함수로 정의

이미지를 input으로 받아서(첫줄에도 보면 image가 argument로 들어감)

중간에 pertrained를 곱해서

마지막에 예상된 분류를 아웃풋으로 return.

In [ ]:
import os
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/tiger.jpg"
os.system("curl " + url + " > tiger.jpg")
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/dog.jpg"
os.system("curl " + url + " > dog.jpg")

두개의 url은 호성좌 깃헙에 있는 이미지. 호랑이와 강아지 그림. 이 url들을 받아서 os.system 함수에 적어주면 > 이하의 이름으로 url 이미지 파일을 콜랩에 다운로드(좌측 상단).

깃헙에서 콜랩으로 가져왔으니...
이후 func함수를 그라디오화할 때 개 그림과 호랑이 그림을 예시로 제시할 것.

In [ ]:
gr.Interface(fn=func, inputs='image', outputs='text', examples = ['tiger.jpg', 'dog.jpg']).launch()

func함수를 정의해줬으니 최종적으로 그라디오화. **입력의 데이터 유형과 출력의 데이터 유형을 정의해줘야함.** 입력은 이미지를 입력하고, 출력은 텍스트로 뱉길 원함. 위에 있는 함수의 입력 유형과 출력유형을 그대로 적어주는 것.


examples는 아까 다운로드받은 두개의 tiger, dog 이미지 파일을 그라디오에서 예시로 띄워주는 것.


**참고로 examples에 들어갈 사진들은 중괄호를 써서 list로 묶어줘야함.**


이렇게 띄워주면 굳이 drag&drop 방식으로 이미지를 가져올 필요 없이 example를 클릭하기만 하면 됨. (외주 맡긴 사장님이 좋아하심)

### [Fill-Mask](https://huggingface.co/tasks/fill-mask)

빈칸 채우기

e.g. https://huggingface.co/bert-base-uncased \
How to use

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("Hello I'm a [MASK] model.")

Hello I'm a [MASK] model 이 문장을 가지고 실험하는 것

결과물은 dictionary 여러개가 list로 묶여있음. 빈칸에 들어갈 적절 단어, full sentence, 적절한 확률 등이 list로 묶여서 표현된 것.

unmasker : mask를 채워주는 함수라는 뜻.

Demo in Gradio

In [ ]:
import pandas as pd
def func (text):
  unmasker = pipeline('fill-mask', model='bert-base-uncased')
  result = unmasker(text)
  df = pd.DataFrame(result)
  return df

먼저 인풋 아웃풋을 정의하는 fuction을 정의. 

모델을 불러와서 unmasker라고 했고, 그 unmasker로부터 text를 받아와서 결과물이라고 정의.

아까 위 함수의 결과값이 list였으므로.. 데이터 프레임으로 전환해서 return

In [ ]:
examples = ["Hello I'm a [MASK] model.", "It is raining outside. I feel [MASK]."]

역시나 예시를 띄어주기 위해 미리 정의. cold, sick 등이 적절한 단어일 것.

In [ ]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()

inputs에는 text를 넣어주고

output은 dataframe형태. -> score(적정률), token_str(적정 단어), sequence(full sentence) 등이 dataframe으로 묶여 나옴.

example을 실행하면 cold, sick 등이 도출됨.

### [Token classification](https://huggingface.co/tasks/token-classification)

natural language understading(자연어 처리)의 일종. 어떤 자연어가 주어져 있다면(문장이나 텍스트), 엄청 자유도가 높을 것. 즉 아무 문장이나 단어나 다 가능. 

그러나 컴퓨터는 자유도가 높은 환경에 적합하지 않음. 따라서 컴퓨터는 여기서 *필요한 단어나 정보만을* 추출. 

Named Entity Recognition이나 Part of Speech tagging(품사 태그)가 그 중 하나의 기술. 특정 중요 명사와 정보를 AI가 캐치해내는 것. 

로케이션 정보(LOC)와 조직/단체 정보(ORG), 혹은 사람에 대한 정보(PER)를 위주로 캐치해냄. MISC는 기타.

예를 들어 my name is Wolfgang and I live in Berlin이라는 문장이 있으면 다른 건 다 버리고 Wolfgang은 PER로 인식되고 Berlin은 LOC로 추출된다. 

e.g. https://huggingface.co/dslim/bert-base-NER \
How to use

In [ ]:
!pip install transformers

recognize four types of entities: location (LOC), organizations (ORG), person (PER) and Miscellaneous (MISC).

자연어에는 많은 정보가 있지만 그 중에서도 위의 4개의 정보가 가장 중요하다.

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

tokenizer : stirng의 형태로 문장을 쪼개주는 pretrained(행렬).단어 수준의 형태로 문장을 쪼갬. 본격적인 처리에 앞서서 전처리라고 할 수 있음.

model : 각각의 string을 classify하는 pretrained.

nlp는 이가 종합된 모델. example로 정의된 문장을 이 모델에 넣고 돌리면 각각의 토큰에 대한 설명이 결과로 도출됨.

Demo in Gradio

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
def func (text):
  tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
  model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
  nlp = pipeline("ner", model=model, tokenizer=tokenizer)
  result = nlp(text)
  df = pd.DataFrame(result)
  return df

함수로 정의하기

In [ ]:
examples = ["My name is Wolfgang and I live in Berlin", "I will visit Seoul to see Chris"]

예시

In [ ]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()

여기서도 output은 dataframe형태로 도출.

entity : LOC인지 ORG인지 PER인지 분류
score: 정확도
word : 해당 단어
등이 dataframe으로 제시됨.


그라디오 돌리면 Berlin과 Seoul이 Loc으로 나오고 Chris와 Wolfgang은 PER로 잘 인식됨. 스코어가 확률값.

### [Sentence similarity](https://huggingface.co/tasks/sentence-similarity)

input text(raw data)를 feature 형태로 바꾸어 vector(**embedding**)로 convert해줌. 
이렇게 바꾸면 두 텍스트들 간에 유사성을 쉽게 계산. 

e.g. https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 \
How to use

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
sentences = ["This is an example sentence", "it is one example writing"]

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(sentences)
print(embeddings)

sentences : 비교할 두 문장이 들어간 리스트

embedding : sentences를 넣어서 숫자로 바꾼 후 list화. 두 개의 list가 묶여서 하나의 list가 되어 있음. 

In [9]:
len(embeddings)

2

오해하면 안 됨. embeddings에 2가지 숫자가 있는게 아니라

2개의 list로 이루어져 있는 것.



In [4]:
len(embeddings[0])

384

embeddings[0] : embeddings를 이루고 있는 리스트 중 첫 번째 list

embeddings[1] : embeddings를 이루고 있는 리스트 중 두 번째 list

len(embeddings[0]) : 첫번째 리스트 안의 숫자 개수 -> 384개

len(embeddings[1]) : 두번째 리스트 안의 숫자 개수 -> 384개

-> 참고로 이 384개의 숫자 vector가 feature vector.

In [ ]:
cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])
cosine_scores

두 개의 embeddings을 비교하는 법 : 각도

예를 들어 (2,1)과 (1,2)를 비교하기 위해선 이를 x,y축에 있는 2차원에 표시한 다음 원점에서 각 점까지의 직선을 그리고 직선 간의 각도 차이로 유사성을 비교.

각도 차이 = 0 <- 유사성 높음
각도 차이 = 90 <- 유사성 낮음

여기에 코사인함수를 넣으면
cos(0) = 1 -> 코사인 = 1 일때 유사성 가장 높음
cos(90) = 0 -> 코사인 = 0 일 때 유사성 가장 낮음

따라서 cosine similarity라고 하는 것.

vector에 384개의 숫자가 있으므로 384차원의 그래프에 점을 그려서 각도 비교하면 됨. -> 그 함수가 바로 util.pytorch.cos_sin

Demo in Gradio

In [6]:
def func (text1, text2):
  from sentence_transformers import SentenceTransformer, util
  model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
  embeddings = model.encode(text1, text2)
  cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])
  return cosine_scores

위의 과정 다 합쳐서 function화

참고) 입력에서 처음부터 text1, text2 넣어주고
embeddings부터 쭉 2개 씩 넣어줌 -> 2개의 list가 합쳐져서 하나의 list화

이후 cosine_socres에서 각도비교

In [7]:
examples = ["My name is Wolfgang and I live in Berlin", "I will visit Seoul to see Chris"]

In [ ]:
gr.Interface(fn=func, inputs=['text', 'text'], outputs='number', examples = examples).launch()

output : 유사도를 숫자 형태로 나타냄